## 오늘 할 내용

- cifar10 이미지 데이터를 사용한 모델링
   - convolution layer (cnn : 합성곱 신경망)
   - 직접 모델링 & 학습 




In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# 자동 손쉬운 변환법
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    # numpy array를 그림으로 바꿔주는 함수
    plt.imshow(img)
    plt.show()

    
for i in range(5):    
    imshow(trainset.data[i])


In [ ]:
trainset.data[0].shape

## Training

In [147]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 3개의 input chnnel, 6개의 convolution filter, 5x5의 kernel size
        self.conv1 = nn.Conv2d(in_channels=3,
                               out_channels=6,
                               kernel_size=5) # 이 표현식과 같음 : kernel_size=(5,5)
        # 2 x 2 max pooling
        self.pool = nn.MaxPool2d(2, 2)
        # input 6, output 16
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        self.fc1 = nn.Linear(16*6*6, 100)
        self.fc2 = nn.Linear(100, 60)
        self.fc3 = nn.Linear(60, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 16*6*6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # 마지막 레이어에선 activation function을 넣지 않음
        x = self.fc3(x)
        
        x = F.softmax(x, dim=1)
        
        return x

### 네트워크가 어떻게 진행되는지 하나씩 실행해보기
- .size()를 통해서 어떤 방식으로 dimension이 변화하는지 check
![image](https://miro.medium.com/max/3294/1*vkQ0hXDaQv57sALXAJquxA.jpeg)

In [105]:
x = trainset.data[0]

x.shape

x = x.reshape(1, 3, 32, 32)
x = torch.tensor(x).float()
x.size()

In [108]:
conv1 = nn.Conv2d(in_channels=3,
                  out_channels=6,
                  kernel_size=5)
conv1


Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))

In [109]:
x = conv1(x)
x.size()

torch.Size([1, 6, 28, 28])

In [110]:
pool = nn.MaxPool2d(2, 2)
pool

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [111]:
x = pool(x)
x.size()

torch.Size([1, 6, 14, 14])

In [112]:
conv2 = nn.Conv2d(6, 16, 3)
conv2

Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))

In [113]:
x = conv2(x)
x.size()

torch.Size([1, 16, 12, 12])

In [114]:
x = pool(x)
x.size()

torch.Size([1, 16, 6, 6])

In [115]:
# 뒷 채널과 맞춰주기 위한 reshape
x = x.view(-1, 16*6*6)
x.size()

torch.Size([1, 576])

In [118]:
fc1 = nn.Linear(16*6*6, 100)
fc1

Linear(in_features=576, out_features=100, bias=True)

In [117]:
x = fc1(x)
x.size()

torch.Size([1, 100])

In [120]:

fc2 = nn.Linear(100, 60)
fc3 = nn.Linear(60, 10)

In [122]:
x = fc2(x)
x = fc3(x)


In [124]:
x.size()
x

tensor([[ 20.9631,  -7.2326,   5.9510,   1.5061,  -2.8366,  -1.8465,  -1.1821,
          -1.0033, -15.1670,  -3.3763]], grad_fn=<AddmmBackward>)

In [126]:
F.softmax(x, dim=1)

tensor([[1.0000e+00, 5.6851e-13, 3.0222e-07, 3.5474e-09, 4.6125e-11, 1.2414e-10,
         2.4124e-10, 2.8848e-10, 2.0365e-16, 2.6886e-11]],
       grad_fn=<SoftmaxBackward>)

### 실제 만든 network에 데이터 태우기

In [138]:
net = Net()

In [139]:
test = trainset.data[0]

test.shape

test = test.reshape(1, 3, 32, 32)
test = torch.tensor(test).float()

In [140]:
test.size()

torch.Size([1, 3, 32, 32])

In [144]:
probs = net(test)

In [145]:
probs.size()

torch.Size([1, 10])

In [146]:
probs

tensor([[1.2640e-02, 5.8436e-04, 3.7844e-04, 1.3553e-03, 3.6439e-02, 9.9904e-02,
         6.0141e-01, 5.5843e-04, 1.6814e-02, 2.2991e-01]],
       grad_fn=<SoftmaxBackward>)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# SGD : Stochastic Gradient Descent
optimizer = optim.SGD(net.parameters(), lr=0.001) 

In [ ]:
# running_loss = 0.0
# for i, data in enumerate(trainloader):
#     inputs, labels = data
    
#     optimizer